# Xgboost y variables categóricas 

Fuente: https://forecastegy.com/posts/xgboost-categorical-variables/

En este ejemplo generamos un modelo de Xgboost que integra variables categoricas, veremos como tratarlas para integrarlas correctamente al modelo

## 1) Introducción 
Trabajar con variables categoricas puede ser algo complicado. Xgboost como todos los modelos de ML al final tienen que trabajar con datos numéricos, por lo que en algún momento debe de haber una tranformación de las variables categóricas a una representación numérica. Esto se vuelve complicado si tienes muchas variables categoricas. Un proceso incorrecto de tranformación de variables categoricas puede generar errores en el modelo. 

**Además algunos métodos de encocing pueden aumentar significativamente la dimensionalidad del dataset, haciendo que el problema crezca rápidamente de tamaño y se haga altamente intensivo en memoria**

En este nootebook se va a abordar este problema.......   
puede ser utilizado tanto para problemas de regresión como de clasificación


## 2) Native Encoding Using The XGBoost Scikit-learn Interface
Desde la versión 1.5 de Xgboost, la librería soporta el uso de variables categoricas directamente en el algoritmo. 

Para usar esta funcionalidad se debe de poner como True el parámetro de **enbale_categorica** en nuestro modelo. Sin embargo, es muy importante que se definanda cuales son las variables categoricas en el pandas DF. 

A continuación se muestra un ejemplo: 

### 2.1) Lectura e inspección de los datos 

In [5]:
import pandas as pd 

# 1) Lectura del dataset 
adult_dataset = pd.read_csv(filepath_or_buffer="data/adult.csv")

# 2) Creamos una lista con las variables categoricas 
cat_cols = ['marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'workclass', 'education']

# 3) Inspección del tipo de variables 
adult_dataset.dtypes 

age                int64
workclass         object
fnlwgt             int64
education         object
education.num      int64
marital.status    object
occupation        object
relationship      object
race              object
sex               object
capital.gain       int64
capital.loss       int64
hours.per.week     int64
native.country    object
income            object
dtype: object

Como se puede observar, el tipo de variables que tengo actualmente  son int (enteros) y objetos (en el contxto de pandas esto lo podemos pensar como un tipo string

### 2.2) Tranformación del tipo de variables 


In [6]:
# 1) Inció de bucle 
for col in cat_cols:
    adult_dataset[col] = adult_dataset[col].astype('category')
    
# 2) Inspección del pandas DF 
adult_dataset.dtypes # Ahora vemos que las variables son tipo categoricas 

age                  int64
workclass         category
fnlwgt               int64
education         category
education.num        int64
marital.status    category
occupation        category
relationship      category
race              category
sex               category
capital.gain         int64
capital.loss         int64
hours.per.week       int64
native.country    category
income              object
dtype: object

Una vez que tus variables categoricas ya están convertidas a tipo categorico en tu pandas DF, podemos ingresar directamente el dataset al XGbost utilizando el parametro de **enable_categorical=True**. 


Cuando utilizas `enable_categorical=True` en XGBoost y pasas un DataFrame de Pandas con columnas categóricas transformadas a tipo "category" usando `.astype("category")`, estás indicando a XGBoost que trate estas columnas de forma especial como características categóricas en lugar de características numéricas.

El parámetro `enable_categorical` fue introducido en XGBoost 1.3.0 y permite al modelo manejar directamente las variables categóricas sin necesidad de preprocesarlas a través de métodos como One-Hot Encoding o Label Encoding. Esto tiene varias ventajas:

1. **Mejor Uso de la Memoria**: El manejo interno de categorías puede ser más eficiente en memoria que la creación de columnas adicionales a través de One-Hot Encoding, especialmente cuando las variables categóricas tienen muchos niveles.

2. **Rendimiento Potencialmente Mejorado**: Al preservar la naturaleza categórica de la característica, XGBoost puede hacer divisiones que son más significativas para las categorías en lugar de tratarlas como valores numéricos arbitrarios.

3. **Menor Complejidad de Preprocesamiento**: No es necesario convertir las variables categóricas en números enteros o binarizarlas, lo que simplifica el flujo de trabajo de preprocesamiento de datos.

Cuando XGBoost se encuentra con una característica de tipo "category" y `enable_categorical=True`, el algoritmo utiliza un método de codificación especial interno para las variables categóricas, que se basa en la técnica de "one-hot-encoding" pero optimizada para trabajar dentro del algoritmo de árboles de decisión de XGBoost. 


### 2.3) Split de datos 
Generaremos un training y un testing DF  

**Generamos nuestro y & X**

In [13]:
# Split into X and y, drop the target variable from X and convert y to binary
y = adult_dataset['income'].map(arg={'<=50K': 0, '>50K': 1}) # La funcion map sustituye valores, en este caso estamos sustituyendo los valores de <=50K por 0 y los valores de >50K por 1. 

X = adult_dataset.drop('income', axis=1)
y 

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    0
32558    1
32559    0
32560    0
Name: income, Length: 32561, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
5514,26,Private,256263,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,25,United-States
19777,24,Private,170277,HS-grad,9,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States
10781,36,Private,75826,Bachelors,13,Divorced,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32240,22,State-gov,24395,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,20,United-States
9876,31,Local-gov,356689,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,25,Private,410240,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States
5390,51,Private,146767,Assoc-voc,11,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
860,55,Federal-gov,238192,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Male,0,1887,40,United-States
15795,41,Private,154076,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,50,United-States


### 2.4) Entrenamiento 

In [17]:
from xgboost import XGBClassifier

# Initialize XGBoost classifier
model = XGBClassifier(enable_categorical=True, tree_method='hist')

# Fit the model
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### 2.5) Evaluación 

In [19]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(X_test)

score = accuracy_score(y_true= y_test , y_pred= y_predict)
score

0.8707200982650084

## 3) Categorical Encoding Using One-Hot Encoding
En este sección haremos nuevamente el entrenamiento del modelo. Sin embago trataremos directamente las variables categoricas con en encoding de one-hot-encoding.

One-hot-encoding es un método popular para la tranformación de variables categoricas, genera columnas que representan cada categoría y le dan un valor binario si el registro pertenece o no a una categoría en particular. Este método es simple y efectivo. Sin embargo, tiene un gran problema y este es que puede aumentar significativamente la dimensionalidad de nuestro dataset

In [25]:
# Instala esta librería especializada en método de codigicación de variables categoricas 
# pip install category_encoders
from category_encoders import OneHotEncoder

# Initialize OneHotEncoder
encoder = OneHotEncoder(cols=cat_cols)

# Fit and transform the DataFrame
df_encoded = encoder.fit_transform(adult_dataset)
df_encoded

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,workclass_8,workclass_9,...,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,native.country_42,income
0,90,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
1,82,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
2,66,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
3,54,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
4,41,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
32557,27,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
32558,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,>50K
32559,58,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K


The best thing about this library is that it returns a DataFrame with both the original numerical columns and the new encoded columns, avoiding an extra step of concatenating the two DataFrames.

In [27]:
# Split into X and y, drop the target variable from X and convert y to binary
y = df_encoded['income'].map(arg={'<=50K': 0, '>50K': 1}) # La funcion map sustituye valores, en este caso estamos sustituyendo los valores de <=50K por 0 y los valores de >50K por 1. 

X = df_encoded.drop('income', axis=1)
X 

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,workclass_8,workclass_9,...,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,native.country_42
0,90,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,82,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,54,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32557,27,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32558,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32559,58,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,workclass_8,workclass_9,...,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,native.country_42
5514,26,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19777,24,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10781,36,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32240,22,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9876,31,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,25,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,51,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,55,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15795,41,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Initialize XGBoost classifier
model = XGBClassifier(enable_categorical=False, tree_method='hist')

# Fit the model
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [31]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(X_test)

score = accuracy_score(y_true= y_test , y_pred= y_predict)
score

0.8702594810379242

In [21]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Crear un DataFrame de ejemplo con dos variables numéricas y una categórica
data = pd.DataFrame({
    'Edad': [25, 30, 35, 40, 45],
    'Ingresos': [50000, 60000, 75000, 80000, 90000],
    'Ciudad': ['Nueva York', 'Los Ángeles', 'Chicago', 'San Francisco', 'Miami']
})

# Crear una instancia de OneHotEncoder
encoder = OneHotEncoder()

# Ajustar y transformar los datos
one_hot_encoded = encoder.fit_transform(data[['Ciudad']])

one_hot_encoded

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [22]:
# La salida es una matriz dispersa (sparse matrix) de tipo CSR (Compressed Sparse Row)
# Puedes convertirla en un DataFrame si lo deseas
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(['Ciudad']))
one_hot_encoded_df

,Ciudad_Chicago,Ciudad_Los Ángeles,Ciudad_Miami,Ciudad_Nueva York,Ciudad_San Francisco
0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0


In [23]:
# Combinar las variables numéricas y las codificadas one-hot en un solo DataFrame
encoded_data = pd.concat([data[['Edad', 'Ingresos']], one_hot_encoded_df], axis=1)
encoded_data

,Edad,Ingresos,Ciudad_Chicago,Ciudad_Los Ángeles,Ciudad_Miami,Ciudad_Nueva York,Ciudad_San Francisco
0,25,50000,0.0,0.0,0.0,1.0,0.0
1,30,60000,0.0,1.0,0.0,0.0,0.0
2,35,75000,1.0,0.0,0.0,0.0,0.0
3,40,80000,0.0,0.0,0.0,0.0,1.0
4,45,90000,0.0,0.0,1.0,0.0,0.0


In [ ]:


# Resultado
print(encoded_data)